In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import tensorflow as tf
from tensorflow import keras


/home/subhasish/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/subhasish/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/subhasish/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/subhasish/anaconda3/lib/python3.6/site-packages/tensorflow/python/f

In [2]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
# import imageio

# Bayesian

In [111]:
def f(x):
    val = 2*x[0]+ x[1]
    return val

In [112]:
#defining the objective function 

def objective(X, target):
    
#     X = torch.tensor(X).float()
#     X = X.cuda()
#     res = net(X) - target
 
    res = f(X) - target
    #return f,net(x),target
    return res 


X_init = [0,1]
Y_init = f(X_init)
target = 7.02

In [113]:
#defining a random dummy hidden constraint function. 
###########################
#TODO @ipsi : Actually we need to use LR modelling
###########################

def hidden(X):
    
    if np.random.uniform(0,1) > 0.5:
        return 1
    else:
        return -1

In [114]:
from scipy.stats import norm

def expected_improvement(X, X_sample, Y_sample, gpr, gpr_h, xi=0.01):
    '''
    Computes the EI at points X based on existing samples X_sample
    and Y_sample using a Gaussian process surrogate model.
    
    Args:
        X: Points at which EI shall be computed (m x d).
        X_sample: Sample locations (n x d).
        Y_sample: Sample values (n x 1).
        gpr: A GaussianProcessRegressor fitted to samples.
        xi: Exploitation-exploration trade-off parameter.
        **kwargs: GPR for modelling the hidden constraint values (arg name : 'gpr_h')
    
    Returns:
        Expected improvements at points X.
    '''
    mu, sigma = gpr.predict(X, return_std=True)
    mu_sample = gpr.predict(X_sample)

    sigma = sigma.reshape(-1, 1)
    
    # Needed for noise-based model,
    # otherwise use np.max(Y_sample).
    # See also section 2.4 in [...]
    mu_sample_opt = np.max(mu_sample)

    #Computing the posterior mean and var for the hidden constraint
    mu_h, sigma_h = gpr_h.predict(X, return_std=True)

    sigma_h = sigma_h.reshape(-1, 1)


    with np.errstate(divide='warn'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
             
        #Compting new acquisition function
        ei = ei * norm.cdf(-mu_h/sigma_h)

        
    return ei

In [115]:
from scipy.optimize import minimize

def propose_location(acquisition, X_sample, Y_sample, gpr, gpr_h, bounds, n_restarts=25):
    '''
    Proposes the next sampling point by optimizing the acquisition function.
    
    Args:
        acquisition: Acquisition function.
        X_sample: Sample locations (n x d).
        Y_sample: Sample values (n x 1).
        gpr: A GaussianProcessRegressor fitted to samples.

    Returns:
        Location of the acquisition function maximum.
    '''
    dim = X_sample.shape[1]
    min_val = 1
    min_x = None
    
    def min_obj(X):
        # Minimization objective is the negative acquisition function
        return -acquisition(X.reshape(-1, dim), X_sample, Y_sample, gpr, gpr_h)
    
#     print(np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, dim)))
    
    # Find the best optimum by starting from n_restart different random points.
    
    for x0 in np.random.uniform(bounds[:, 0], bounds[:,1],size=(n_restarts, dim)):
        
        #print(len(x0))
        res = minimize(min_obj, x0=x0, bounds=bounds, method='L-BFGS-B')        
        if res.fun < min_val:
            min_val = res.fun[0]
            min_x = res.x           
            
    return min_x.reshape(-1, 1)

In [116]:
import numpy as np

In [117]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, Matern


In [118]:
def bayes_opt(model,x_init, y, h, bounds, n_iter = 100):
    '''
    model : the output from the neural network 
    x (list):the first 8 variables whose values are fixed
    y (list): the target value 
    bounds (list of list): lower and upper bounds of the variables whose values are to be determined 
    '''
    target = y[0]
    
    #model.eval()

    # Gaussian process with Matérn kernel as surrogate model
    m52 = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5)
    gpr = GaussianProcessRegressor(kernel=m52)
    gpr_h = GaussianProcessRegressor(kernel=m52)

    # Initialize samples
    x_sample = np.array(x_init).reshape(1, -1)
    y_sample = np.array(y)
    h_sample = np.array(h)
    

    for i in range(n_iter):
        # Update Gaussian process with existing samples
        gpr.fit(x_sample, y_sample)
        gpr_h.fit(x_sample, h_sample)

        # Obtain next sampling point from the acquisition function (expected_improvement)
        x_next = propose_location(expected_improvement, x_sample, y_sample, gpr, gpr_h, bounds)
        x_next = x_next.reshape(1,-1)
        x_next = list(x_next[0])
        x = x_next
        #x.extend(x_next)                        this commented out part was for the case where the first few x_i in
#                                                a_1x_1+a2x_2 +...... = y was fixed.


        # Obtain next noisy sample from the objective function
        y_next = objective(x, target)
        y_next = y_next.tolist()
        
        # Obtain next hidden constraint 
        h_next = hidden(x)
        #h_next = h_next.tolist()
        
        # Add sample to previous samples
        x_sample = np.vstack((x_sample, x_next))
        y_sample = np.vstack((y_sample, y_next))
        h_sample = np.vstack((h_sample, h_next))
        
        print('New sampled input : {}, observed output {}, constraint {}'.format(x_next, y_next, h_next))
        tolerance = 0.5
        change = np.linalg.norm(x_sample[len(x_sample)-1]-x_sample[len(x_sample)-2],2 )
        #print(change)
        
        ######################################
        #TODO @ipsi : We have a budget n_iter = 100, why are we stopping before that ?
        ######################################
        
        if change < tolerance:
            
            print('\nAfter {} iterations, minimum objective value achieved {}'.format(i, np.min(y_sample)))
            print('\nOptimal input {}'.format(x_sample[np.argmin(y_sample)]))
            break
#         if i != n_iter-1:                      this commented out part was for the case where the first few x_i in
#                                                a_1x_1+a2x_2 +...... = y was fixed.

#             x = x[:-3]
                

    #return x 
    
    ######################################
    #TODO @ipsi : why r u returning x? shouldn't it be like this 
    ######################################
    
    return x_sample[np.argmin(y_sample)]

In [119]:
######################################
#TODO @ipsi : Only "one" (n_init = 1) observation to start with ?
######################################

x_init = [0,1]
y_init = [7.04]
h_init = [1]

# on changing the number of variables it will automatically detect from the dimension of the bounds
limits = np.array([[-2,2],[-2,2]])

In [120]:
output = bayes_opt(f, x_init, y_init, h_init, limits, 100)

New sampled input : [-1.1452246285278662, 2.0], observed output -7.330449257055733, constraint -1
New sampled input : [-1.5510894170811027, 0.7078919463349713], observed output -9.434286887827234, constraint 1
New sampled input : [-1.1444176732450624, 2.0], observed output -7.328835346490125, constraint -1
New sampled input : [-1.2901742783990087, 2.0], observed output -7.620348556798017, constraint -1

After 3 iterations, minimum objective value achieved -9.434286887827234

Optimal input [-1.55108942  0.70789195]


In [121]:
output

array([-1.55108942,  0.70789195])